<a href="https://www.kaggle.com/code/milkbottledude/titanical-machine-learning-mark-ii?scriptVersionId=207883348" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Data Configuration


In [2]:
# Train Data Configuration
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.drop(['Name', 'PassengerId'], axis=1, inplace=True)
train_data['TotalRelatives'] = train_data['SibSp'] + train_data['Parch']
age_mean = train_data['Age'].mean()
train_data['Age'] = train_data['Age'].fillna(age_mean)
X = pd.get_dummies(train_data.drop('Survived', axis=1))
y = train_data["Survived"]
print(X)

     Pclass        Age  SibSp  Parch     Fare  TotalRelatives  Sex_female  \
0         3  22.000000      1      0   7.2500               1       False   
1         1  38.000000      1      0  71.2833               1        True   
2         3  26.000000      0      0   7.9250               0        True   
3         1  35.000000      1      0  53.1000               1        True   
4         3  35.000000      0      0   8.0500               0       False   
..      ...        ...    ...    ...      ...             ...         ...   
886       2  27.000000      0      0  13.0000               0       False   
887       1  19.000000      0      0  30.0000               0        True   
888       3  29.699118      1      2  23.4500               3        True   
889       1  26.000000      0      0  30.0000               0       False   
890       3  32.000000      0      0   7.7500               0       False   

     Sex_male  Ticket_110152  Ticket_110413  ...  Cabin_F G73  Cabin_F2  \


In [3]:
# Test Data Configuration
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
PassengerId_Series = test_data.PassengerId
test_data.drop(['Name', 'PassengerId'], axis=1, inplace=True)
test_data['TotalRelatives'] = test_data['SibSp'] + test_data['Parch']
test_age_mean = test_data['Age'].mean()
fare_median = test_data['Fare'].median()
test_data['Age'] = test_data['Age'].fillna(test_age_mean)
test_data['Fare'] = test_data['Fare'].fillna(fare_median)
X_test_fr = pd.get_dummies(test_data)
print(X_test_fr)

     Pclass       Age  SibSp  Parch      Fare  TotalRelatives  Sex_female  \
0         3  34.50000      0      0    7.8292               0       False   
1         3  47.00000      1      0    7.0000               1        True   
2         2  62.00000      0      0    9.6875               0       False   
3         3  27.00000      0      0    8.6625               0       False   
4         3  22.00000      1      1   12.2875               2        True   
..      ...       ...    ...    ...       ...             ...         ...   
413       3  30.27259      0      0    8.0500               0       False   
414       1  39.00000      0      0  108.9000               0        True   
415       3  38.50000      0      0    7.2500               0       False   
416       3  30.27259      0      0    8.0500               0       False   
417       3  30.27259      1      1   22.3583               2       False   

     Sex_male  Ticket_110469  Ticket_110489  ...  Cabin_F E46  Cabin_F E57 

Gonna try to fix the problem of missing ticket and cabin column names between the testing and training datasets below

In [4]:

SetX = set(X.columns)
SetTestX = set(X_test_fr.columns)
ToRemove = list(SetTestX - SetX)
# print(ToRemove)
ToAdd = list(SetX - SetTestX)
X_test_fr.drop(ToRemove, axis=1, inplace=True)
ToAdd_df = pd.DataFrame(False, index=X.index, columns=ToAdd)
X_test_fr = pd.concat([X_test_fr, ToAdd_df], axis=1)
X_test_fr = X_test_fr.iloc[:418]

# Get the column order of X
column_order = X.columns
# Reordering X_test_fr to match the column order of X
X_test_fr = X_test_fr[column_order]
rows_with_nan = X_test_fr[X_test_fr.isna().any(axis=1)]

# RandomForestClassifier

In [5]:
# from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

model = RandomForestClassifier(random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


model.fit(X_train, y_train)
predictions = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, predictions))

Test Accuracy: 0.8547486033519553


# Logistic Regression (TBC)

In [6]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score
# import matplotlib.pyplot as plt
# from sklearn.model_selection import GridSearchCV


# model = LogisticRegression(random_state=0)
# model.fit(X, y)


# Submission

In [7]:
model = RandomForestClassifier(bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100)
model.fit(X, y)
pred = model.predict(X_test_fr)


# Submission
output = pd.DataFrame({'PassengerId': PassengerId_Series, 'Survived': pred})
print(output)
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
Your submission was successfully saved!
